In [42]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os 
import json
import cv2

import xml.etree.ElementTree as ET
from tqdm import tqdm

In [ ]:
# Final Data Spec: {breed_name: [{file_name:{bnd_box_dict}}]}
# Example: {'n02085620-Chihuahua': [{'n02085620_10074': {'xmin': 71,
                                            #      'ymin': 1,
                                            #      'xmax': 192,
                                            #      'ymax': 180}}]    

## TODO:

* load
* cut bb from image
* resize to standard
* loose final channel
* data albumentation
* normalize the dataset

* prepare dataset

* upload to colab to train model using cnn
* deploy model and upload on github

In [62]:
data = {}

for breed in tqdm(os.listdir("./stanford-dogs/images"), desc="Collecting data..."): ##each breed
    images = os.listdir(f"./stanford-dogs/images/{breed}")
    annotations = os.listdir(f"./stanford-dogs/annotation/{breed}")
    for img_filename, annt_filename in zip(images,annotations): ## iterate over images in each breed
        bounds = []
        for annt_filename in annotations:
            bounds_dict = {}
            with open(f"./stanford-dogs/annotation/{breed}/{annt_filename}", "r") as f:
                tree = ET.ElementTree(ET.fromstring(f.read()))
                for tag in ["xmin", "ymin", "xmax", "ymax"]:
                    bounds_dict[tag] = int(tree.getroot().find("object").find("bndbox").find(tag).text)
            bounds.append({annt_filename : bounds_dict})
    data[breed] = bounds   

In [64]:
with open("data-spec.json", "w") as f:
    json.dump(data, f)

In [114]:
dim = 128
data_dict = {}

for breed, imgs in tqdm(data.items()):
    breed_resized = []
    for img_dict in imgs:
        for img_name,bndbox_dict in img_dict.items():
            img_array = plt.imread(f"./stanford-dogs/images/{breed}/{img_name}.jpg")
            if img_array.shape[2] ==3  :
                roi = img_array[bndbox_dict["ymin"]:bndbox_dict["ymax"], bndbox_dict["xmin"]:bndbox_dict["xmax"], :]    
                resized_roi = cv2.resize(roi, (128,128), interpolation = cv2.INTER_LINEAR) ##image size: (dim,dim,3)
                breed_resized.append(resized_roi)
    data_dict[breed] = breed_resized

100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [02:19<00:00,  1.17s/it]


In [149]:
import pickle

with open("model_data.pkl", "wb") as f:
    pickle.dump(data_dict, f)